In [110]:
%%time
# Expiry, symbol and option info extracted from NSE
# Takes 10 minutes

import requests
import lxml.html as lh
import datetime as dt
import pandas as pd
import numpy as np
import csv
import os

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'

#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        pass

#******               Symbols list          ****
#________________________________________________

eq_symbols='ACC,ADANIENT,ADANIPORTS,ADANIPOWER,AJANTPHARM,ALBK,AMARAJABAT,AMBUJACEM,ANDHRABANK,APOLLOHOSP,\
APOLLOTYRE,ARVIND,ASHOKLEY,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,\
BALKRISIND,BALRAMCHIN,BANKBARODA,BANKINDIA,BATAINDIA,BEML,BERGEPAINT,BEL,BHARATFIN,BHARATFORG,\
BPCL,BHARTIARTL,INFRATEL,BHEL,BIOCON,BOSCHLTD,BRITANNIA,CADILAHC,CANFINHOME,CANBK,CAPF,CASTROLIND,\
CEATLTD,CENTURYTEX,CESC,CGPOWER,CHENNPETRO,CHOLAFIN,CIPLA,COALINDIA,COLPAL,CONCOR,CUMMINSIND,DABUR,\
DALMIABHA,DCBBANK,DHFL,DISHTV,DIVISLAB,DLF,DRREDDY,EICHERMOT,ENGINERSIN,EQUITAS,ESCORTS,EXIDEIND,\
FEDERALBNK,GAIL,GLENMARK,GMRINFRA,GODFRYPHLP,GODREJCP,GODREJIND,GRANULES,GRASIM,GSFC,HAVELLS,HCLTECH,\
HDFCBANK,HDFC,HEROMOTOCO,HEXAWARE,HINDALCO,HCC,HINDPETRO,HINDUNILVR,HINDZINC,ICICIBANK,ICICIPRULI,IDBI,\
IDEA,IDFCBANK,IDFC,IFCI,IBULHSGFIN,INDIANB,IOC,IGL,INDUSINDBK,INFIBEAM,INFY,INDIGO,IRB,ITC,JISLJALEQS,\
JPASSOCIAT,JETAIRWAYS,JINDALSTEL,JSWSTEEL,JUBLFOOD,JUSTDIAL,KAJARIACER,KTKBANK,KSCL,KOTAKBANK,KPIT,\
L%26TFH,LT,LICHSGFIN,LUPIN,M%26MFIN,MGL,M%26M,MANAPPURAM,MRPL,MARICO,MARUTI,MFSL,MINDTREE,MOTHERSUMI,\
MRF,MCX,MUTHOOTFIN,NATIONALUM,NBCC,NCC,NESTLEIND,NHPC,NIITTECH,NMDC,NTPC,ONGC,OIL,OFSS,ORIENTBANK,PAGEIND,\
PCJEWELLER,PETRONET,PIDILITIND,PEL,PFC,POWERGRID,PTC,PNB,PVR,RAYMOND,RBLBANK,RELCAPITAL,RCOM,RELIANCE,\
RELINFRA,RPOWER,REPCOHOME,RECLTD,SHREECEM,SRTRANSFIN,SIEMENS,SREINFRA,SRF,SBIN,SAIL,STAR,SUNPHARMA,SUNTV,\
SUZLON,SYNDIBANK,TATACHEM,TATACOMM,TCS,TATAELXSI,TATAGLOBAL,TATAMTRDVR,TATAMOTORS,TATAPOWER,TATASTEEL,TECHM,\
INDIACEM,RAMCOCEM,SOUTHBANK,TITAN,TORNTPHARM,TORNTPOWER,TV18BRDCST,TVSMOTOR,UJJIVAN,ULTRACEMCO,UNIONBANK,\
UBL,MCDOWELL-N,UPL,VEDL,VGUARD,VOLTAS,WIPRO,WOCKPHARMA,YESBANK,ZEEL'.split(',')

idx_symbols=['NIFTY', 'BANKNIFTY']

#......      Data Limiter   .......
# symbols = eq_symbols+idx_symbols
symbols = eq_symbols+idx_symbols
#..................................

# generic url for equity and index options
url_base = "https://nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol="
url_end = '&date='

def get_opt_data(symbol):
    '''Gets the options data
    Arg: (symbol) as str
    Returns: scraped dataframe'''
    
#     symbol = 'NIFTY'
    url = url_base + symbol
    
    while_flag = True   # Flag to loop over expiries for a symbol
    while_counter = 0   # Counter for expiry
    expiry = ''         # Initialize expiry for first run
    df = pd.DataFrame() # Initialize return dataframe

    while while_flag:

        # If it is not first run, append next expiry
        if expiry != '':  # This is not the first run
            try:
                expiry = expiry[while_counter]
                url = url_base+symbol+url_end+expiry  # Check to see if this works
            except IndexError:
                while_flag = True   # Get out of the while loop
                break

        # Get raw data from the url
        page = requests.get(url)
        doc = lh.fromstring(page.content)

        #...    Get the Expiries ...#
        #____________________________

        the_path= "//form"   # Get the form content. There are two forms
        expiries = doc.xpath(the_path)[0].text_content()
        strip_chars = [ord('\n'), ord('\xa0'), ord('\t'), ord('\r')] # characters to be stripped
        char_table = {s: ' ' for s in strip_chars} # table for translate to locate the chars

        sym_exp = expiries.translate(char_table).split() # split converts the translated dict into list

        # Get the expiries only, from the table
        expiry = [sym_exp[k+i] 
         for k, v in enumerate(sym_exp) 
         if v in 'Select' 
         for i in range(len(sym_exp) - k)][1:]

        # Convert expiry to datetime
        expiry_dt = [dt.datetime.strptime(date, "%d%b%Y").date() for date in expiry]

        #...   Get the Options Data .... #
        #_________________________________

        tbl_path = "//*[@id='octable']"  # xpath for the table

        # The headers
        opt_head = ['cOI', 'cChnginOI', 'cVolume', 'cIV', 'cLTP', 
                    'cNetChng', 'cBidQty', 'cBidPrice', 'cAskPrice', 'cAskQty', 'Strike', 
                    'pBidQty', 'pBidPrice', 'pAskPrice', 'pAskQty', 'pNetChng', 'pLTP', 
                    'pIV', 'pVolume', 'pChnginOI', 'pOI']

        # Extract the rows within the header in the table
        opt_data = [[tr.text_content().strip() 
                    for table in doc.xpath(tbl_path)] 
                    for th in table.xpath('//thead') 
                    for tr in th.xpath('//tr')][5:]  # First 5 rows are junk

        df1 = pd.DataFrame(data=[a.translate(char_table).split() 
                                    for d in opt_data for a in d], columns=opt_head)

        df2 = df1.replace(',', '', regex=True) # Remove comma from numbers
        df2 = df2.apply(pd.to_numeric, errors='coerce') # Convert to numeric
        df2.insert(0, 'Expiry', expiry_dt[while_counter]) # Insert the Expiry column
        df2.insert(0, 'Symbol', symbol) # Insert the Symbol

        # Rearrange the columns
        cols_beginning = ['Symbol', 'Expiry', 'Strike']
        df2 = df2[cols_beginning + [c for c in df2 if c not in cols_beginning]]
        while_counter = while_counter + 1
        df = df.append(df2)
        return df

# %%time
nse_options = [catch(lambda: get_opt_data(symbol)) for symbol in symbols]

df_options = pd.concat(nse_options).reset_index(drop=True)

#****        Closing tasks     ****
#___________________________________

# write list of successful symbols to a csv file - for the next program

def write_csv(the_list, filenm):
    '''Writes to the file as a csv
    Args: 
       (the_list) as list
       (filenm) as the name of file, with or without extension
    Returns: None'''
    
    # Check if file_name has an extension
    filenm = filenm.split('.')[0]
    
    if os.path.exists(r'./zdata/'+filenm+r'.csv'):
        print(filenm+'.csv'+' exists and is overwritten!')
    
    with open(datapath+filenm+r'.csv', 'w', newline='') as csvfile:
        wr = csv.writer(csvfile)
        wr.writerow(the_list)

good_idx = list(set(list(df_options.Symbol.unique())) & set(idx_symbols))
good_equity = list(set(list(df_options.Symbol.unique())) & set(eq_symbols))

write_csv(filenm='nse_idx_symbols', the_list=good_idx)
write_csv(filenm='nse_eq_symbols', the_list=good_equity)

# write to pickle file - for the next program   
df_options.to_pickle('./zdata/nse_options.pkl')   # Pickle the dataframe for later use

Wall time: 0 ns


In [119]:
df_options

,Symbol,Expiry,Strike,cOI,cChnginOI,cVolume,cIV,cLTP,cNetChng,cBidQty,...,pBidQty,pBidPrice,pAskPrice,pAskQty,pNetChng,pLTP,pIV,pVolume,pChnginOI,pOI
0,ACC,2018-10-25,9100.0,NaN,NaN,NaN,NaN,NaN,NaN,150.0,...,30975.0,3.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACC,2018-10-25,9150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3000.0,3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACC,2018-10-25,9200.0,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,...,8100.0,7.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACC,2018-10-25,9250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6825.0,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACC,2018-10-25,9300.0,NaN,NaN,NaN,NaN,NaN,NaN,75.0,...,75.0,17.05,18.90,150.0,16.80,17.05,29.76,1002.0,49800.0,49800.0
5,ACC,2018-10-25,9350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3150.0,8.00,NaN,NaN,NaN,3.05,NaN,NaN,NaN,75.0
6,ACC,2018-10-25,9400.0,3000.0,NaN,NaN,NaN,1188.40,NaN,75.0,...,225.0,20.15,21.00,1800.0,20.95,21.00,28.74,2391.0,76650.0,76650.0
7,ACC,2018-10-25,9450.0,47025.0,-600.0,14.0,NaN,864.00,-279.40,75.0,...,75.0,22.20,24.30,75.0,11.60,22.40,27.96,2306.0,1425.0,95400.0
8,ACC,2018-10-25,9500.0,874500.0,37275.0,2195.0,NaN,811.15,-300.75,900.0,...,6975.0,27.35,27.90,75.0,15.20,27.55,28.16,36524.0,432375.0,1374525.0
9,ACC,2018-10-25,9550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,75.0,24.05,NaN,NaN,6.40,22.40,25.51,118.0,4800.0,5025.0
